In [ ]:
shap_valuess=pd.DataFrame()
shap_valuess1=pd.DataFrame()#

for model_idx in Classifiers_best: #param_best:
    model  = param_best[model_idx]
    if model_idx !='DummyClassifier':
        print(model_idx)
        my_model = Classifiers_best[model_idx].fit(X_train,Y_train)
        explainer = shap.TreeExplainer(my_model)
        shap_values = explainer.shap_values(X_test)
        shap_valuess[model_idx]=shap_values
        shap_valuess1[model_idx]=shap_values[1].ravel()
      


In [ ]:
def modeliser_data_unbalanced(nsplits,X_train, Y_train,X_test,Y_test,Classifiers):

  best_params= {}
  cout_append= []
  frames_test= [] 
  score_value= []
  scores = []
  column_Classifiers=[]
  save_model = pd.DataFrame()
  threshold=[0,0.001,0.002, 0.003,0.004,0.005,0.006,0.007,0.008,0.009,0.01,0.011, 
             0.012, 0.013,0.014,0.015, 0.02,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]

  frames_FI=pd.DataFrame()
  frames_FI1=pd.DataFrame()
  for model_idx in Classifiers:
    column_Classifiers.append(model_idx)
    print(model_idx)
    model  = Classifiers[model_idx]
    my_model=model.fit(X_train, Y_train)
    #   
    Findex      = str('C.Pred')      +'_' + model_idx
    Findex_proba= str('C.Pred_proba')+'_' + model_idx  
    y_predite   = pd.DataFrame(my_model.predict(X_test) ,index=Y_test.index,columns=[Findex]) 
    y_pred_proba= pd.DataFrame(my_model.predict_proba(X_test)[:,1] ,index=Y_test.index,columns=[Findex_proba])
    
    if model_idx == 'DummyClassifier':
      y_pred      = pd.concat([Y_test,y_predite, y_pred_proba],axis=1)
    else:
      y_pred      = pd.concat([y_predite, y_pred_proba],axis=1)
    frames_test.append(y_pred)

    # Check Feature Importance
    if model_idx != 'DummyClassifier':
      explainer  = shap.TreeExplainer(my_model)
      shap_values= explainer.shap_values(X_test)
      frames_FI[model_idx]=shap_values
      frames_FI1[model_idx]=shap_values[1].ravel()

    # Function cout
    acu_scores= accuracy_score(Y_test, y_predite)
    auc_scores= roc_auc_score(Y_test, y_pred_proba) 
    data      = {'Model':model_idx,'ACCURACY': [acu_scores], 'ROC_AUC':[auc_scores]}  
    scores.append(pd.DataFrame(data) )

    cost_function = []
    cost_ind = []
    for thre in threshold:
      Ypred=(y_pred_proba>thre)
      Ypred = np.array(Ypred > 0) * 1
      cost_function.append(custom_metric(Y_test, Ypred))
      cost_ind.append(thre)
      newname=str('cout') + '_'+ str(model_idx)
    cout    = pd.DataFrame(cost_function,columns=[newname]) 
    cout_append.append(cout)
    if model_idx == 'DummyClassifier':
      idx_cout= pd.DataFrame(cost_ind,columns=['Threshold']) 

    df_pred = pd.concat(frames_test, axis=1)
    
    cost    = pd.concat(cout_append,axis=1)
    cost['Threshold']= idx_cout['Threshold']
    cost      = cost.set_index('Threshold')
    df_results= pd.concat(scores,axis=0)
    df_results= df_results.set_index('Model')
    save_model[model_idx]=[my_model]

  return df_pred, frames_FI, frames_FI1, df_results, cost, save_model


In [ ]:


Notons un fort déséquilibre entre la précision trouvée pour la Target 0 (0.92) et la Target 1 (0). Pour rappel, l'échantillon de travail n'est pas équilibré, avec 92% des individus classés en modalité 0 et 8% en modalité 1, à savoir en défaut de paiement de crédit..

Il est donc intéressant de travailler cet Oversampling (ou suréchantillonnage en français) en ajustant la distribution de classe de manière à avoir une répartition plus égalitaire.
